## Importing and setting up paths

In [50]:
import sys,os,importlib,gc
import xarray as xr
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

os.chdir('/home/peter/Projects/tc_emulator/results')

sys.path.append('../scripts')
import _weather_pattern_class; importlib.reload(_weather_pattern_class)

atl = _weather_pattern_class.weather_patterns(source='ERA5', working_directory='./')

## Prepare weather patterns

### prepare input

here the raw data is loaded and some preprocessing is applied:

* select ASO
* compute standardized anomalies
* regrid to 2.5x2.5 over 90W-10W and 10N-30N

In [51]:
# load MSLP and VWS
# these fields were preprocessed using the scripts in input_data_preparation
tmp = xr.load_dataset('/home/peter/Projects/data/ERA5/ERA5_vws_atl_1x1_daily_rmTC.nc')['vws'].loc['1982':'2020']
atl.add_data(tmp, 'VWS', 'PRGn_r')
tmp = xr.load_dataset('/home/peter/Projects/data/ERA5/ERA5_MSLP_atl_1x1_daily_rmTC.nc')['MSLP'].loc['1982':'2020']
atl.add_data(tmp, 'MSLP', 'RdBu_r')

In [52]:
# the input is transformed to standardized anomalies and only the ASO season is considered
atl.preprocess_select_months(months=[8,9,10], month_colors=['#bfef45','#ffe119','#f58231'])
atl.preprocess_standardized_anomalies(['1982','2011'])
atl.preprocess_regrid(reference_lats=np.arange(10, 30, 2.5),reference_lons=np.arange(-90, -10, 2.5))
atl.preprocess_create_vector(variables = ['VWS','MSLP'])
atl.store_input('ERA5_VWS-MSLP_noTC3')

/home/peter/miniconda3/envs/py_main/lib/python3.9/site-packages/xarray/core/dataarray.py:789: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return key in self.data


### load input

In [53]:
atl.load_input('ERA5_VWS-MSLP_noTC3')
years = np.array(range(1982,2021))
atl.set_split(years=years)

### SOM weather patterns

Weather patterns are obtained from **S**elf **O**rganizing **M**aps \

In [74]:
from minisom import MiniSom
from sklearn.metrics import pairwise_distances

def SOM_(X):
    som = MiniSom(nrows,ncols,X.shape[1], random_seed=seed, sigma=1,learning_rate=0.01)
    som.pca_weights_init(atl._vector.values)
    som.train(X, 1000000, verbose=False)
    out = {'centers_map':som.get_weights().reshape((nrows*ncols,X.shape[1])), 'other_results':{'SOM':som}}
    return out

nrows,ncols = 5,4
seed = 1
tag = 'SOM_pcaInit%sx%s_v%s' % (nrows,ncols,seed)
atl.mapping_raw()
atl.load_mapping('mapping_raw')
atl.clustering_wrapper(SOM_, tag=tag, overwrite=True)

#### Load weather patterns

In [75]:
tag = 'SOM_pcaInit%sx%s_v%s' % (nrows,ncols,seed)
atl.define_plot_environment(pre_mapping='mapping_raw', clustering=tag, post_mapping='mapping_sammon_1982-2020', nrows=nrows, ncols=ncols)
atl.stats_TC(file='tracks/tracks_ibtracks.csv', overwrite=True)

{'SOM': <minisom.MiniSom object at 0x7fc058b97130>}
.//ERA5_VWS-MSLP_noTC3/mapping_raw_1982-2020/SOM_pcaInit5x4_v1/mapping_sammon_1982-2020/grid_5x4


In [76]:
atl.plot_fields()
atl.stats_frequency()
atl.plot_freq()
atl.plot_label_trend()

# scatter plot
atl.plot_events(indicator='wind', legend_values=[64,96,137], legend_name='wind speed [kn]')
atl.plot_events(indicator='ACE')

# lagged stats
atl.plot_stats(indicator='ACE')
atl.plot_stats(indicator='genesis')

/home/peter/Projects/tc_emulator/scripts/_weather_pattern_class.py:932: MatplotlibDeprecationWarning: normalize=None does not normalize if the sum is less than 1 but this behavior is deprecated since 3.3 until two minor releases later. After the deprecation period the default value will be normalize=True. To prevent normalization pass normalize=False 
  wedges, autotexts = ax.pie(self._stats['freq_dec'].loc[lab].values, colors=self._decades['dec_colors'], startangle=90, radius=self._stats['count'].loc[lab].values)
